In [1]:
#import requests
#import pandas as pd
#from bs4 import BeautifulSoup #use if doing static sites
import selenium
from selenium import webdriver
import os
import time
from PIL import Image
import io
import hashlib
import requests

In [3]:
#setting up the webdriver
DRIVER_PATH = "C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/chromedriver_win32/chromedriver.exe"
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

C:\Users\catri\AppData\Local\Temp\ipykernel_15260\3206594383.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [3]:
#driver test - loading google
wd.get('https://google.com')

In [5]:
#testing driver - entering "dog" into search bar
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Dogs')

C:\Users\catri\AppData\Local\Temp\ipykernel_58864\4188249259.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = wd.find_element_by_css_selector('input.gLFyf')


In [4]:
#quitting test driver
wd.quit()

In [5]:
#function for searching google for images (query=what to enter in search bar)
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [8]:
#function for saving the images we get
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [6]:
#combining the search and save functions
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [22]:
#test saving images 
driver_path = DRIVER_PATH
target_path = "C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/images"
number_images = 5

search_term_1 = "dirty plastic cups"
search_term_3 = "dirty plastic bottles"
search_term_5 = "dirty plastic cutlery"

#cups - dirty
search_and_download(search_term_1, driver_path,target_path,number_images)
#bottles - dirty
search_and_download(search_term_3, driver_path,target_path,number_images)
#cutlery - dirty
search_and_download(search_term_5, driver_path,target_path,number_images)

C:\Users\catri\AppData\Local\Temp\ipykernel_58864\4091825908.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
C:\Users\catri\AppData\Local\Temp\ipykernel_58864\1550939233.py:20: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\catri\AppData\Local\Temp\ipykernel_58864\1550939233.py:34: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 5 image links, done!
SUCCESS - saved https://thumbs.dreamstime.com/b/empty-dirty-disposable-plastic-cup-empty-dirty-disposable-plastic-cup-isolated-white-background-180772907.jpg - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi week start 04.04/Projets/Projet P8 - Triof/triof/images\dirty_plastic_cups\5bd84380bf.jpg
SUCCESS - saved https://thumbs.dreamstime.com/b/empty-dirty-disposable-plastic-cup-isolated-white-background-empty-dirty-disposable-plastic-cup-180772948.jpg - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi week start 04.04/Projets/Projet P8 - Triof/triof/images\dirty_plastic_cups\7001e539a3.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/close-empty-used-coffee-cups-260nw-53005576.jpg - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi week start 04.04/Projets/Projet P8 - Triof/triof/images\dirty_plastic_cups\204bbd0e42.jpg
SUCCESS - saved https://thumbs.dreamstime.com/z/empty-dirty-disposable-plastic-cup-empty-dirty-disposable-plastic-cup-isolated

In [9]:
#creating the training dataset
driver_path = DRIVER_PATH
target_path_training = "C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/training_images"
number_images = 100

search_term_1 = "dirty plastic bottles"
search_term_2 = "dirty plastic cups"
search_term_3 = "dirty plastic cutlery"

#dirty cottles
search_and_download(search_term_1, driver_path,target_path_training,number_images)
#dirty cups
search_and_download(search_term_2, driver_path,target_path_training,number_images)
#dirty cutlery
search_and_download(search_term_3, driver_path,target_path_training,number_images)

C:\Users\catri\AppData\Local\Temp\ipykernel_15260\4091825908.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
C:\Users\catri\AppData\Local\Temp\ipykernel_15260\1550939233.py:20: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\catri\AppData\Local\Temp\ipykernel_15260\1550939233.py:34: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 100 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQSELXdRr9B4fFvbM3J6OWita4YPc1YIYWfoQ&usqp=CAU - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/training_images\dirty_plastic_bottles\31bfb9a531.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQn1HN4nVGOZIIG7n7DidKqUzO2W0QsSTIggQ&usqp=CAU - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/training_images\dirty_plastic_bottles\2df374d1d3.jpg
SUCCESS - saved https://thumbs.dreamstime.com/b/dirty-plastic-bottle-isolated-black-background-dirty-plastic-bottle-water-isolated-black-background-115917145.jpg - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/training_images\dirty_plastic_bottles\eb8322e8a6.jpg
SUCCESS - saved https://media.gettyimages.com/photos/plastic-bottles-are-seen-in-the-river-taff-on-sep

In [10]:
#creating the test dataset
#used French to try and get different photos - would need another technique IRL
driver_path = DRIVER_PATH
target_path_test = "C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/test_images"
number_images = 30

search_term_1 = "bouteilles en plastique sales"
search_term_2 = "gobelets en plastique sales"
search_term_3 = "couverts en plastique sales"

#bouteilles en plastique sales
search_and_download(search_term_1, driver_path,target_path_test,number_images)
#cups - clean
search_and_download(search_term_2, driver_path,target_path_test,number_images)
#bottles - dirty
search_and_download(search_term_3, driver_path,target_path_test,number_images)

C:\Users\catri\AppData\Local\Temp\ipykernel_15260\4091825908.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
C:\Users\catri\AppData\Local\Temp\ipykernel_15260\1550939233.py:20: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\catri\AppData\Local\Temp\ipykernel_15260\1550939233.py:34: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 31 image links, done!
SUCCESS - saved https://i0.wp.com/lecorpslamaisonlesprit.fr/wp-content/uploads/2015/06/COMMENT-NETTOYER-UNE-BOUTEILLE.png - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/test_images\bouteilles_en_plastique_sales\350a732764.jpg
SUCCESS - saved https://images.freeimages.com/images/premium/previews/1588/15885570-still-life-of-a-dirty-crushed-plastic-water-bottle.jpg - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/test_images\bouteilles_en_plastique_sales\c788484a36.jpg
SUCCESS - saved https://thumbs.dreamstime.com/z/vieille-bouteille-en-plastique-sale-73326132.jpg - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/test_images\bouteilles_en_plastique_sales\8fd5870528.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRX9jNFIBfJtUEZLj_ZVR5ONpRuh4omMs0hMQ&usqp=CAU - as C:/Use

C:\Users\catri\anaconda3\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


SUCCESS - saved https://www.ecoverre.com/images/branding/easybox-avantatges.png - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/test_images\gobelets_en_plastique_sales\93fb584558.jpg
SUCCESS - saved https://www.delidrinks.com/16416-large_default/gobelet-plastique-rpet-16oz473ml-o95mm-x-50.jpg - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/test_images\gobelets_en_plastique_sales\4244f2c7ec.jpg
SUCCESS - saved https://www.lafoirfouille.fr/medias/sys_master/images/images/hb3/hd4/8865972224030/10000021312-0-1200Wx1200H.jpg - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/test_images\gobelets_en_plastique_sales\86ea4e1837.jpg
SUCCESS - saved https://www.robe-materiel-medical.com/images_produits/zoom/image_399_gobe30t_-_gobelets_gradus_30ml.jpg?1650361790 - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof proj

In [17]:
#creating the training dataset for just clean and dirty
driver_path = DRIVER_PATH
target_path_training = "C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/cleandirty_training"
number_images = 150

search_term_1 = "dirty plastic"
search_term_2 = "plastic cups, plastic bottles, plastic cutlery"

#dirty plastic
search_and_download(search_term_1, driver_path,target_path_training,number_images)
#clean plastic
search_and_download(search_term_2, driver_path,target_path_training,number_images)

C:\Users\catri\AppData\Local\Temp\ipykernel_15260\4091825908.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
C:\Users\catri\AppData\Local\Temp\ipykernel_15260\1550939233.py:20: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\catri\AppData\Local\Temp\ipykernel_15260\1550939233.py:34: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 151 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRkGt42rlZJa9MrlYGn1EE9mGret4G79LAZig&usqp=CAU - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/cleandirty_training\dirty_plastic\b54bcf9bb0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR1whUM9cd2neSGBld1QlQ3oDSqxWZAr7m3vQ&usqp=CAU - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/cleandirty_training\dirty_plastic\9183278216.jpg
SUCCESS - saved https://cdn.w600.comps.canstockphoto.com/empty-used-dirty-glass-and-plastic-pictures_csp86926985.jpg - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/cleandirty_training\dirty_plastic\505bf8e2e9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQTGNUDoGjFv0cXCVK8kUPAQftL4W3SwF7bhA&usqp=CAU - as C:/Users/catri/OneDrive/Simplon/IA-P2-

In [18]:
#creating the test dataset for just clean and dirty
driver_path = DRIVER_PATH
target_path_training = "C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/cleandirty_test"
number_images = 50

search_term_1 = "gobelets en plastique, bouteilles en plastique, couverts en plastique"
search_term_2 = "déchets plastiques"

#dirty plastic
search_and_download(search_term_1, driver_path,target_path_training,number_images)
#clean plastic
search_and_download(search_term_2, driver_path,target_path_training,number_images)

C:\Users\catri\AppData\Local\Temp\ipykernel_15260\4091825908.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
C:\Users\catri\AppData\Local\Temp\ipykernel_15260\1550939233.py:20: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\catri\AppData\Local\Temp\ipykernel_15260\1550939233.py:34: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 51 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQqmiFyMi9XHYBlR18jQK4cUsLxpHsc7NSvQQ&usqp=CAU - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/cleandirty_test\gobelets_en_plastique,_bouteilles_en_plastique,_couverts_en_plastique\721982255c.jpg
SUCCESS - saved https://lebonemballage.com/2131-thickbox_default/bouteille-plastique.jpg - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/cleandirty_test\gobelets_en_plastique,_bouteilles_en_plastique,_couverts_en_plastique\43052e215d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQstDqI-v9f2wJJRJsyJ1hnFYDUxIvFGkSBp4lyC6AONegCS2N0-JqtODjlpBJyz05TCdg&usqp=CAU - as C:/Users/catri/OneDrive/Simplon/IA-P2-Euskadi with my Triof project/Projets/Projet P8 - Triof/triof/cleandirty_test\gobelets_en_plastique,_bouteilles_en_plastique,_couverts_en_plastique\483db725bd.jpg

In [11]:
#Louie version
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import shutil
import os
import sys
import argparse
from binascii import a2b_base64
from tqdm import tqdm

def parse_arguments():
    parser = argparse.ArgumentParser(description='Un petit scraper pour Google Image')
    parser.add_argument('type',
                        help='type of object to search for, this will be used as the directory name and as a name stem for image storage')
    parser.add_argument('search',
                        help='Google search for which you want to get the images')
    parser.add_argument('-n', '--number', type=int, default=10,
                        help='Number of images you want to store')
    parser.add_argument('-d', '--directory', default='scraped_images',
                        help='The parent directory in which to store the files, images will be stored in a subdirectory named after the type parameter')
    
    args = parser.parse_args()

    return args.type, args.search, args.number, args.directory

def scrap_imgs(obj_type, search, number=10, directory='scraped_images'):

    print("Connection et recherche google:\n...")
    opts = webdriver.FirefoxOptions()
    opts.add_argument("--headless")
    driver = webdriver.Firefox(options=opts)

    store_dir = os.path.join(directory, obj_type)

    if not os.path.exists(directory):
        os.makedirs(directory)

    if not os.path.exists(store_dir):
        os.makedirs(store_dir)
        i_max=0
    else:
        i_max = max(map(lambda name : int(name.split('_')[1].split('.')[0]),
                    os.listdir('scraped_images/dirty')))

    url = f'https://www.google.com/search?q={search}&source=lnms&tbm=isch&sa=X&ved=2ahUKEwie44_AnqLpAhUhBWMBHUFGD90Q_AUoAXoECBUQAw&biw=1920&bih=947'

    driver.get(url)
    
    #on scroll pour pouvoir scraper plus d'images
    print("Scrolling de la page de résultats de Google Image:")
    for _ in tqdm(range(10)):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);")
        time.sleep(0.5)
        print("Récupération des images:")
        cpt=0
        flag=False
        for i in tqdm(range(number)):
            img_src = driver.find_element(By.XPATH, '//div//div//div//div//div//div//div//div//div//div['+str(i+1)+']//a[1]//div[1]//img[1]').get_attribute("src")
            filename = f'/{obj_type}_{i_max+i+1}.jpg'

            if img_src is None:
                cpt+=1
                flag=True

            else:
                if img_src[:10] == 'data:image':
                    data = img_src.split(',')[1]
                    binary_data = a2b_base64(data)
                    with open(store_dir + filename, 'wb') as f:
                        f.write(binary_data)

                elif img_src[:4] == 'http':
                    response = requests.get(img_src,stream=True)
                    with open(store_dir + filename, 'wb') as f:
                        shutil.copyfileobj(response.raw, f)

        if flag:
            print(f"*** !! ATTENTION !! ***\n{cpt} image(s) n'a(ont) pas pu être scrappée(s)")

def main():
    args = parse_arguments()
    scrap_imgs(*args)

if __name__ == "__main__":
    main()
        

usage: ipykernel_launcher.py [-h] [-n NUMBER] [-d DIRECTORY] type search
ipykernel_launcher.py: error: the following arguments are required: search


SystemExit: 2

C:\Users\catri\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
